### Dependencies

In [2]:
import pandas as pd

# ETL Project

## Import Database 2 - World Development Indicators

### Objective

Understand which indicatores are consistent in all reported coutries across the years

### Coding

In [3]:
#Import Information

indicators_df = pd.read_csv('indicators.csv')
indicators_df.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.335609e+02
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.779760e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.102333e+01
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3.000000e+06


In [4]:
#Database Inital Evaluation

indicators_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5656458 entries, 0 to 5656457
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   CountryName    object 
 1   CountryCode    object 
 2   IndicatorName  object 
 3   IndicatorCode  object 
 4   Year           int64  
 5   Value          float64
dtypes: float64(1), int64(1), object(4)
memory usage: 258.9+ MB


In [5]:
indicators_df.describe()

,Year,Value
count,5.656458e+06,5.656458e+06
mean,1.994464e+03,1.070501e+12
std,1.387895e+01,4.842469e+13
min,1.960000e+03,-9.824821e+15
25%,1.984000e+03,5.566242e+00
50%,1.997000e+03,6.357450e+01
75%,2.006000e+03,1.346722e+07
max,2.015000e+03,1.103367e+16


In [14]:
#Evaluate Duplicates
indicators_df = indicators_df.drop_duplicates()

In [15]:
#Database Inital Evaluation

indicators_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5656458 entries, 0 to 5656457
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   CountryName    object 
 1   CountryCode    object 
 2   IndicatorName  object 
 3   IndicatorCode  object 
 4   Year           int64  
 5   Value          float64
dtypes: float64(1), int64(1), object(4)
memory usage: 302.1+ MB


In [16]:
indicators_df.describe()

,Year,Value
count,5.656458e+06,5.656458e+06
mean,1.994464e+03,1.070501e+12
std,1.387895e+01,4.842469e+13
min,1.960000e+03,-9.824821e+15
25%,1.984000e+03,5.566242e+00
50%,1.997000e+03,6.357450e+01
75%,2.006000e+03,1.346722e+07
max,2.015000e+03,1.103367e+16


In [10]:
#New dataset to address consistency of data (Country, Year, Indicator)
data_df = indicators_df.copy(['CountryCode','Year','IndicatorCode'])

In [11]:
#Count number of different Countries
number_countries = len(data_df['CountryCode'].unique())
number_countries

247

In [12]:
#Count number of different indicators
number_indicators = len(data_df['IndicatorCode'].unique())
number_indicators

1344

In [13]:
#Count number of different Years
number_year = len(data_df['Year'].unique())
number_year

56

In [17]:
#Evaluate consistency of data across Indicators/Years/Countries - how many countries has reported which indicators by year
data_consistency = pd.DataFrame(data_df.groupby(['IndicatorCode','Year'])['CountryCode'].count())

In [18]:
data_consistency

CountryCode
IndicatorCode            Year             
AG.AGR.TRAC.NO           1961          187
                         1962          187
                         1963          187
                         1964          187
                         1965          187
...                                    ...
per_si_allsi.cov_pop_tot 2010           25
                         2011           24
                         2012           22
                         2013            4
                         2014            1

[45373 rows x 1 columns]

In [20]:
#Evaluate indicators reported consistently in all data countries
data_consistency.loc[data_consistency['CountryCode']==number_countries]

CountryCode
IndicatorCode  Year             
AG.SRF.TOTL.K2 2011          247
               2012          247
               2013          247
               2014          247
SP.POP.GROW    1999          247
               2000          247
               2002          247
               2003          247
               2004          247
               2005          247
               2006          247
               2008          247
               2009          247
               2010          247
               2013          247
               2014          247
SP.POP.TOTL    1998          247
               1999          247
               2000          247
               2001          247
               2002          247
               2003          247
               2004          247
               2005          247
               2006          247
               2007          247
               2008          247
               2009          247
               2010          247
               2011          247
               2012          247
               2013          247
               2014          247